LangChain Expression Language is a way to create arbitrary custom chains. It is built on the Runnable protocol.

One point about LangChain Expression Language is that any two runnables can be "chained" together into sequences. The output of the previous runnable's .invoke() call is passed as input to the next runnable. This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

The resulting RunnableSequence is itself a runnable, which means it can be invoked, streamed, or further chained just like any other runnable. Advantages of chaining runnables in this way are efficient streaming (the sequence will stream output as soon as it is available), and debugging and tracing with tools like LangSmith.

The pipe operator: |
To show off how this works, let's go through an example. We'll walk through a common pattern in LangChain: using a prompt template to format input into a chat model, and finally converting the chat message output into a string with an output parser.

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_together import ChatTogether
import os

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= os.getenv("TOGETHER_API_KEY")
)

In [3]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke about football."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'setup': 'Why did the football go to the doctor?',
 'punchline': 'Because it was feeling a little deflated!'}

In [4]:
from langchain_community.callbacks import get_openai_callback

In [5]:
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(result.content)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you laugh! Do you want to hear another one?
---

Total Tokens: 71
Prompt Tokens: 39
Completion Tokens: 32
Total Cost (USD): $0.0
